In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from pprint import pprint
from datetime import datetime

In [21]:
def scrape_reviews(company, pagecount, english = False):
    df = pd.DataFrame()
    if english:
        base_url = 'https://www.trustpilot.com/review/'
    else:
        base_url = 'https://de.trustpilot.com/review/'
    for i in range(1,pagecount+1):
        url = base_url+company+'?page='+str(i)
        response = requests.get(url)
        if response.status_code != 200:
            print('Keine weitere Seite gefunden.')
            break
        else:
            reviews = []
            ratings = []
            date = []
            soup = BeautifulSoup(response.content, "html.parser")
            script_tag = soup.find('script', type='application/ld+json', attrs={'data-business-unit-json-ld': 'true'})
            if script_tag:
                try:
                    json_data = json.loads(script_tag.string)
                except json.JSONDecodeError:
                    print("Fehler beim Decodieren der JSON-Daten.")
                else:
                    for element in (json_data['@graph']):
                        if element['@type'] == 'Review':
                            reviews.append(element['reviewBody'])
                            ratings.append(element['reviewRating']['ratingValue'])
                            datum_alt = datetime.strptime(element['datePublished'], "%Y-%m-%dT%H:%M:%S.%fZ")
                            datum_neu = datum_alt.strftime("%Y-%m-%d")
                            date.append(datum_neu)
                    review_df = pd.DataFrame({'reviews': reviews, 'ratings': ratings, 'date': date})
                    df = pd.concat([df, review_df], axis=0, ignore_index=True)
            else:
                print("Kein <script> Tag mit den gewünschten JSON-Daten gefunden.")
    return df

In [22]:
scrape_reviews('www.airbnb.de', 1, english=False)

,reviews,ratings,date
0,Diskriminierung meiner Person Unterstellung da...,1,2024-07-19
1,Mir ist aufgefallen dass ich seit ein paar Mon...,1,2024-07-20
2,Kunden Support gleich null. Ich kann nicht meh...,1,2024-07-14
3,Am liebsten würde ich 0 Sterne geben.Mein Gast...,1,2024-07-16
4,Uns ging es wie all den anderen Bewertern hier...,1,2024-07-14
5,Habe heute versucht für 2025 ein Appartement a...,1,2024-07-16
6,Die angebotenen Wohnungen oder Zimmer sind nur...,1,2024-07-13
7,Wir sind zur Zeit in Norwegen. Bei unserem vor...,1,2024-07-12
8,AirBnB ist für mich Geschichte. Solange der Wo...,1,2024-07-11
9,Ich wollte eine Unterkunft mieten. Es hat nur ...,1,2024-07-09
